In [22]:

from collections import deque
from six import next
import readers


import tensorflow as tf
import numpy as np


import time

In [23]:
np.random.seed(42)#指定随机种子,方便复现

u_num = 6040 #用户数
i_num = 3952 #电影数

batch_size = 1000 #批尺寸，每次训练部分
dims = 5 #暂时指定隐含因子的纬度 m*5 5*n
max_epochs = 50 #1个epoch等于使用训练集中的全部样本训练一次；

place_device = "/cpu:0"


In [24]:
def get_data():
    df = read_file("./ml-1m/ratings.dat",sep="::")
    rows = len(df)
    
    #重定位进行乱序洗牌操作，消失用户顺序的影响
    df = df.iloc[np.random.permutation(rows)].reset_index(drop=True)
    
    #指定训练集，测试集的比例 ，0.9训练
    splite_index = int(rows * 0.9)
    df_train = df[0:splite_index]
    df_test = df[splite_index:].reset_index(drop=True)
    
    return df_train, df_test

def clip(x):
    return np.clip(x,1.0,5.0)

In [73]:
def model(user_batch, item_batch, user_num, item_num, dim=5, device="/cpu:0"):
    with tf.device("/cpu:0"):
        # 全局大环境
        bias_global = tf.get_variable("bias_global", shape=[])
        # 用户与电影偏差
        w_bias_user = tf.get_variable("embd_bias_user", shape=[user_num])
        w_bias_item = tf.get_variable("embd_bias_item", shape=[item_num])
        # 训练其中一部分 ，参数定义100个，不会再更新
        bias_user = tf.nn.embedding_lookup(w_bias_user, user_batch, name="bias_user")
        bias_item = tf.nn.embedding_lookup(w_bias_item, item_batch, name="bias_item")

        w_user = tf.get_variable("embd_user", shape=[user_num, dim],
                                 initializer=tf.truncated_normal_initializer(stddev=0.02))
        w_item = tf.get_variable("embd_item", shape=[item_num, dim],
                                 initializer=tf.truncated_normal_initializer(stddev=0.02))

        embd_user = tf.nn.embedding_lookup(w_user, user_batch, name="embedding_user")
        embd_item = tf.nn.embedding_lookup(w_item, item_batch, name="embedding_item")
    
    with tf.device(device):

        infer = tf.reduce_sum(tf.mul(embd_user, embd_item), 1) #矩阵乘积
        infer = tf.add(infer, bias_global)
        infer = tf.add(infer, bias_user)
        infer = tf.add(infer, bias_item, name="svd_inference")
        # l2正则化惩罚项，防止过拟合
        regularizer = tf.add(tf.nn.l2_loss(embd_user), tf.nn.l2_loss(embd_item), 
                             name="svd_regularizer")
    return infer, regularizer

In [95]:
#损失函数 = 正则loss，data_loss
def loss(infer,regularizer,rate_batch,learning_rate=0.001, reg=0.1,device="/cpu:0"):
    with tf.device(device):
        #真实值与结果值平方项差异,data_loss
        cost_l2 = tf.nn.l2_loss(tf.subtract(infer,rate_batch))
        #正则化惩罚力度
        penalty = tf.constant(reg, dtype=tf.float32,shape=[],name="l2")
        #组装损失函数
        cost = tf.add(cost_l2,tf.multiply(regularizer,penalty))
        #梯度下降优化器
        train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    return cost, train_op

In [96]:
df_train, df_test = get_data()

samples_per_batch = len(df_train)
print("训练集:%d , 测试集:%d, 迭代次数:%d" % (len(df_train),len(df_test),samples_per_batch))

Number of train samples 900188, test samples 100021, samples per batch 900


In [97]:
print(df_train["user"].head()) 
print(df_test["user"].head())

0    1463
1    1260
2    1205
3    4657
4    5604
Name: user, dtype: int32
0    4138
1    5233
2    1419
3    2019
4    2786
Name: user, dtype: int32


In [98]:
print(df_train["item"].head())
print(df_test["item"].head())

0    2114
1    1209
2     328
3     615
4    2500
Name: item, dtype: int32
0    1403
1    3741
2    1731
3    2114
4    3104
Name: item, dtype: int32


In [99]:
print(df_train["rate"].head())
print(df_test["rate"].head())

0    2.0
1    5.0
2    4.0
3    2.0
4    4.0
Name: rate, dtype: float32
0    3.0
1    5.0
2    5.0
3    3.0
4    3.0
Name: rate, dtype: float32


In [104]:
# 洗牌格式化数据
iter_train = readers.ShuffleIterator([df_train["user"],
                                     df_train["item"],
                                     df_train["rate"]],
                                     batch_size=batch_size)

# S测试数据
iter_test = readers.OneEpochIterator([df_test["user"],
                                     df_test["item"],
                                     df_test["rate"]],
                                     batch_size=-1)

user_batch = tf.placeholder(tf.int32, shape=[None], name="id_user")
item_batch = tf.placeholder(tf.int32, shape=[None], name="id_item")
rate_batch = tf.placeholder(tf.float32, shape=[None])

#运行
infer, regularizer = model(user_batch, item_batch, user_num=u_num, item_num=i_num, dim=dims, device=place_device)
_, train_op = loss(infer, regularizer, rate_batch, learning_rate=0.10, reg=0.05, device=place_device)

In [105]:
#保存模型
saver = tf.train.Saver()
#全局变量初始化
init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    print("%s\t%s\t%s\t%s" % ("Epoch", "Train Error", "Val Error", "Elapsed Time"))
    errors = deque(maxlen=samples_per_batch)
    start = time.time()
    for i in range(max_epochs * samples_per_batch):
        users, items, rates = next(iter_train)
        _, pred_batch = sess.run([train_op, infer], feed_dict={user_batch: users,
                                                               item_batch: items,
                                                               rate_batch: rates})
        pred_batch = clip(pred_batch)
        errors.append(np.power(pred_batch - rates, 2))
        if i % samples_per_batch == 0:
            train_err = np.sqrt(np.mean(errors))
            test_err2 = np.array([])
            for users, items, rates in iter_test:
                pred_batch = sess.run(infer, feed_dict={user_batch: users,
                                                        item_batch: items})
                pred_batch = clip(pred_batch)
                test_err2 = np.append(test_err2, np.power(pred_batch - rates, 2))
            end = time.time()
            
            print("%02d\t%.3f\t\t%.3f\t\t%.3f secs" % (i // samples_per_batch, train_err, np.sqrt(np.mean(test_err2)), end - start))
            start = end

    saver.save(sess, './save/model')

Epoch	Train Error	Val Error	Elapsed Time
00	2.817		1.114		0.034 secs
01	1.047		1.003		1.028 secs
02	0.982		0.968		1.038 secs
03	0.955		0.950		1.011 secs
04	0.941		0.940		1.005 secs
05	0.932		0.934		1.092 secs
06	0.927		0.929		1.317 secs
07	0.923		0.926		1.175 secs
08	0.918		0.923		1.117 secs
09	0.916		0.921		1.013 secs
10	0.914		0.919		1.019 secs
11	0.911		0.918		1.032 secs
12	0.910		0.917		1.037 secs
13	0.909		0.917		1.235 secs
14	0.908		0.915		1.156 secs
15	0.907		0.914		1.420 secs
16	0.907		0.914		1.324 secs
17	0.905		0.914		1.134 secs
18	0.904		0.914		1.020 secs
19	0.904		0.913		1.022 secs
20	0.904		0.913		1.065 secs
21	0.903		0.912		1.005 secs
22	0.902		0.912		1.006 secs
23	0.903		0.911		1.020 secs
24	0.902		0.911		1.036 secs
25	0.901		0.911		1.071 secs
26	0.902		0.912		1.014 secs
27	0.900		0.911		0.994 secs
28	0.901		0.911		1.014 secs
29	0.902		0.910		1.007 secs
30	0.902		0.911		1.046 secs
31	0.901		0.910		0.996 secs
32	0.899		0.910		0.996 secs
33	0.900		0.910		1.010 secs
34	0.89